# IMDB Demo

In [10]:
include("imdb.jl")
using Knet,WordTokenizers
BATCHSIZE=64
SEED=1311194
MAXLEN=150 #maximum size of the word sequence
MAXFEATURES=30000 #vocabulary size
atype = gpu()<0 ? Array{Float32}:KnetArray{Float32}

Array{Float32,N} where N

In [12]:
(xtrn,ytrn,xtst,ytst,imdbdict)=imdb(maxlen=MAXLEN,maxval=MAXFEATURES,seed=SEED)
imdbarray = invert(imdbdict)

88584-element Array{String,1}:
 "the"        
 "and"        
 "a"          
 "of"         
 "to"         
 "is"         
 "br"         
 "in"         
 "it"         
 "i"          
 "this"       
 "that"       
 "was"        
 ⋮            
 "airsoft"    
 "urrrghhh"   
 "ev"         
 "chicatillo" 
 "transacting"
 "sics"       
 "wheelers"   
 "pipe's"     
 "copywrite"  
 "artbox"     
 "voorhees'"  
 "'l'"        

In [13]:
rnd = rand(1:length(xtrn))
println("Sample review:\n",join(imdbarray[xtrn[rnd]]," "),"\n")
println("Classification: ",join(ytrn[rnd]))

Sample review:
reilly reilly reilly reilly reilly reilly reilly 5hrs to start with i have to point out the fact that you're gonna feel completely lost for more than half an hour yeah some things happen but you don't know why or what for when you finally figure things out you just realize that it's nothing but a twisted spastic opera dealing with mature prostitutes dead mothers illegitimate sons the characters are rather poor and the actors specially the young ones don't help that much to spastic look credible only marisa spastic stands out but she's a superb actress no matter if the movie is pure rubbish br br the only positive things to say about spastic sol de spastic is that spastic pablo spastic seems to have good intentions and he's filmed a couple of scenes that are quite intense well maybe the next time br br my rate 4 10

Classification: 1


In [4]:
weights,rnnSpec = loadmodel()

INFO: Loading IMDB...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 75 14.6M   75 11.1M    0     0  1579k      0  0:00:09  0:00:07  0:00:02 2028k

((Float32[0.0568302]

Float32[-0.0575665]

Float32[0.00153457]

...

Float32[0.0121047]

Float32[0.0185812]

Float32[0.0225038], Float32[0.00572032 0.00885967 … -0.0121872 -0.00279316; -0.0130193 -0.00620011 … -0.00328332 0.000816555; … ; -0.00441956 -0.00785721 … -0.000621977 -0.000765511; 0.000269642 -0.00722035 … -0.00257939 0.00205344], Float32[0.149794 -0.149855 … -0.0685552 -0.187412; -0.113529 0.100171 … 0.184712 0.0283851]), Knet.RNN(125, 100, 1, 0.0, 0, 0, 2, 0, Float32, nothing, nothing, nothing, nothing, nothing))

In [5]:
if !(typeof(first(weights)) <: atype)
    weights= map(atype,weights)
end

100 14.6M  100 14.6M    0     0  1843k      0  0:00:08  0:00:08 --:--:-- 2806k


In [6]:
userinput = readline(STDIN)
words = tokenize(lowercase(userinput))
ex = [get!(imdbdict,wr,length(imdbdict)) for wr in words]

stdin> I do not like


4-element Array{Int32,1}:
 10
 78
 21
 37

In [8]:
y = predict(weights,[ex],rnnSpec;train=false)
category = indmax(Array(y))

1